In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def phases(iq: np.ndarray,start:int):
    list = []
    for i in range(start,len(iq)-2,6):
        list.append(
            np.angle(complex(iq[i],iq[i+1]))
        )
    return np.array(list)

antenna_conf= "2-2-0-5-6"

data = pd.read_json("../folder/data.json")
data = data[data["Switch Order"] == antenna_conf]
#data = data[data["TAG"] != "B"]
#data = data[data["rssi"] >-70]
del data["level_0"]
del data["index"]
data=data.reset_index()

In [10]:
tdata = []
tlabels = []

for _, row in data.iterrows():
    for i in range(16,len(row["iq"])-6,3*2):
        tdata.append(row["iq"][i:i+6])
        tlabels.append(row["Angle"])
len(tdata) == len(tlabels)

True

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#Loading the Iris dataset

label_mapper = lambda x: {"0":0,
            "30":1,
                "60":2,
                "90":3,
                "120":4,
                "150":5,
                "180":6,
                }.get(str(x))

X, y = tdata, [label_mapper(x) for x in tlabels]

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True)

#Training the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [18]:
#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names={"0","30","60","90","120","150","180"}))

ValueError: Found input variables with inconsistent numbers of samples: [32558, 1]

In [23]:
prediction = model.predict([X_test[0]])
print({ "0":0,
            "1":30,
            "2":60,
            "3":90,
            "4":120,
            "5":150,
            "6":180}.get(str(prediction[0])))

180


In [19]:
antenna_conf= "14-14-13-8-10"

data = pd.read_json("../folder/data.json")
data = data[data["Switch Order"] == antenna_conf]

vdata = []
vlabels = []

for _, row in data.iterrows():
    for i in range(16,len(row["iq"])-6,3*2):
        vdata.append(np.array(row["iq"][i:i+6]))
        vlabels.append(row["Angle"])

vX, vy = vdata, [label_mapper(x) for x in vlabels]

#Making predictions on the test set
predictions = model.predict(vX)

#Calculating accuracy
accuracy = accuracy_score(vy, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(vy, predictions, target_names={"0","30","60","90","120","150","180"}))


Accuracy: 0.9170264651794995

Classification Report:
              precision    recall  f1-score   support

          90       0.89      0.91      0.90     12716
           0       0.88      0.89      0.89     12672
         180       0.95      0.92      0.94     14113
          60       0.94      0.92      0.93     15345
          30       0.90      0.92      0.91     15466
         150       0.92      0.93      0.93     14256
         120       0.93      0.91      0.92     15488

    accuracy                           0.92    100056
   macro avg       0.92      0.92      0.92    100056
weighted avg       0.92      0.92      0.92    100056



# Real Time

In [44]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import datetime
from scipy.stats import mode


# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
def on_message(client, userdata, msg):
    
    data = json.loads(msg.payload)

    predictions =[]
    for i in range(16,len(data[1]["aoa"]["iq"])-6,3*2):
        sample =data[1]["aoa"]["iq"][i:i+6]
        sample= np.array(sample)
        predictions.append(model.predict([sample]))

    prediction = mode(predictions,axis=None)[0][0]
    print({ "0":0,
            "1":30,
            "2":60,
            "3":90,
            "4":120,
            "5":150,
            "6":180}.get(str(prediction)))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)

now = datetime.datetime.now()
client.loop_forever()


Conectado com o código de resultado 0


C:\Users\pedro\AppData\Local\Temp\ipykernel_14296\2998292462.py:24: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  prediction = mode(predictions,axis=None)[0][0]


1
30
3
90
3
90
1
30
1
30
1
30


KeyboardInterrupt: 

C:\Users\pedro\AppData\Local\Temp\ipykernel_14296\3706323639.py:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode([0,0,1,3],axis=None)[0][0]


0

In [31]:
np.mode()

array([0, 0, 0, 0, 0, 0, 1], dtype=int64)